In [1]:
#importing all important libraries
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import numpy as np, pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,GridSearchCV
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#fifa data set to be used
training_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv')
testing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')

<ipython-input-2-8b8b16b05353>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')


## **1. Data Preparation and Feature Extraction**

Training Data preprocessing

In [3]:
# These are columns that contain ids and url links.
training_data.drop('sofifa_id', axis = 1, inplace = True)
training_data.drop('player_url', axis = 1, inplace = True)
training_data.drop('player_face_url', axis = 1, inplace = True)
training_data.drop('club_logo_url', axis = 1, inplace = True)
training_data.drop('club_flag_url', axis = 1, inplace = True)
training_data.drop('nation_logo_url', axis = 1, inplace = True)
training_data.drop('nation_flag_url', axis = 1, inplace = True)
training_data.drop('nationality_id', axis = 1, inplace = True)
training_data.drop('nation_team_id', axis = 1, inplace = True)
training_data.drop('club_team_id', axis = 1, inplace = True)
training_data.drop('dob', axis = 1, inplace = True) #dropped dob because it basically does the same work as age

In [4]:
# drop columns with 30% or more missing data
missing = (training_data.isnull().sum() / len(training_data)) * 100
columns_missing = missing[missing >= 30].index
training_data.drop(columns=columns_missing, inplace =True)

In [5]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  object 
 1   long_name                    18944 non-null  object 
 2   player_positions             18944 non-null  object 
 3   overall                      18944 non-null  int64  
 4   potential                    18944 non-null  int64  
 5   value_eur                    18707 non-null  float64
 6   wage_eur                     18719 non-null  float64
 7   age                          18944 non-null  int64  
 8   height_cm                    18944 non-null  int64  
 9   weight_kg                    18944 non-null  int64  
 10  club_name                    18719 non-null  object 
 11  league_name                  18719 non-null  object 
 12  league_level                 18719 non-null  float64
 13  club_position   

In [6]:
Y = pd.DataFrame()
Y = training_data['overall'] # separate the overall column as the Y
training_data.drop('overall', axis = 1, inplace = True) # drop overall from the training data
Y

0        93
1        92
2        91
3        91
4        91
         ..
18939    47
18940    47
18941    47
18942    47
18943    47
Name: overall, Length: 18944, dtype: int64

In [7]:
#grouped columns that are numeric
numeric_training = training_data.select_dtypes(include=['int', 'float']).columns # extract numeric data
numeric_training = training_data[numeric_training]
numeric_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   potential                    18944 non-null  int64  
 1   value_eur                    18707 non-null  float64
 2   wage_eur                     18719 non-null  float64
 3   age                          18944 non-null  int64  
 4   height_cm                    18944 non-null  int64  
 5   weight_kg                    18944 non-null  int64  
 6   league_level                 18719 non-null  float64
 7   club_jersey_number           18719 non-null  float64
 8   club_contract_valid_until    18719 non-null  float64
 9   weak_foot                    18944 non-null  int64  
 10  skill_moves                  18944 non-null  int64  
 11  international_reputation     18944 non-null  int64  
 12  release_clause_eur           17949 non-null  float64
 13  pace            

In [8]:
#impute the numeric data using median
imp = SimpleImputer(strategy = 'median')
numeric_training = pd.DataFrame(imp.fit_transform(numeric_training), columns=numeric_training.columns)
numeric_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   potential                    18944 non-null  float64
 1   value_eur                    18944 non-null  float64
 2   wage_eur                     18944 non-null  float64
 3   age                          18944 non-null  float64
 4   height_cm                    18944 non-null  float64
 5   weight_kg                    18944 non-null  float64
 6   league_level                 18944 non-null  float64
 7   club_jersey_number           18944 non-null  float64
 8   club_contract_valid_until    18944 non-null  float64
 9   weak_foot                    18944 non-null  float64
 10  skill_moves                  18944 non-null  float64
 11  international_reputation     18944 non-null  float64
 12  release_clause_eur           18944 non-null  float64
 13  pace            

In [9]:
#grouped all columns with non-numeric values and are objects

object_training = training_data.select_dtypes(exclude= ['int', 'float']).columns # extract the object data
object_training = training_data[object_training]
object_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   club_name         18719 non-null  object
 4   league_name       18719 non-null  object
 5   club_position     18719 non-null  object
 6   club_joined       17961 non-null  object
 7   nationality_name  18944 non-null  object
 8   preferred_foot    18944 non-null  object
 9   work_rate         18944 non-null  object
 10  body_type         18944 non-null  object
 11  real_face         18944 non-null  object
 12  ls                18944 non-null  object
 13  st                18944 non-null  object
 14  rs                18944 non-null  object
 15  lw                18944 non-null  object
 16  lf                18944 non-null  object
 17  cf          

In [10]:
#imputed the non numeric values using most_frequent ie. to fill in with the most appearing
imp = SimpleImputer(strategy='most_frequent') # impute the object data
object_training = pd.DataFrame(imp.fit_transform(object_training), columns = object_training.columns)
object_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   club_name         18944 non-null  object
 4   league_name       18944 non-null  object
 5   club_position     18944 non-null  object
 6   club_joined       18944 non-null  object
 7   nationality_name  18944 non-null  object
 8   preferred_foot    18944 non-null  object
 9   work_rate         18944 non-null  object
 10  body_type         18944 non-null  object
 11  real_face         18944 non-null  object
 12  ls                18944 non-null  object
 13  st                18944 non-null  object
 14  rs                18944 non-null  object
 15  lw                18944 non-null  object
 16  lf                18944 non-null  object
 17  cf          

In [11]:
#encode the object data using the label encoder
encoded_object = pd.DataFrame() # encode the object data
for c in object_training.columns:
  label_encoder = LabelEncoder()
  encoded_object[c] = label_encoder.fit_transform(object_training[c])
encoded_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   short_name        18944 non-null  int64
 1   long_name         18944 non-null  int64
 2   player_positions  18944 non-null  int64
 3   club_name         18944 non-null  int64
 4   league_name       18944 non-null  int64
 5   club_position     18944 non-null  int64
 6   club_joined       18944 non-null  int64
 7   nationality_name  18944 non-null  int64
 8   preferred_foot    18944 non-null  int64
 9   work_rate         18944 non-null  int64
 10  body_type         18944 non-null  int64
 11  real_face         18944 non-null  int64
 12  ls                18944 non-null  int64
 13  st                18944 non-null  int64
 14  rs                18944 non-null  int64
 15  lw                18944 non-null  int64
 16  lf                18944 non-null  int64
 17  cf                18944 non-nul

In [12]:
# combine numeric column and object column with overall
training_data = pd.DataFrame()
training_data = pd.concat([encoded_object, numeric_training], axis = 1)
training_data['overall'] = Y
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  int64  
 1   long_name                    18944 non-null  int64  
 2   player_positions             18944 non-null  int64  
 3   club_name                    18944 non-null  int64  
 4   league_name                  18944 non-null  int64  
 5   club_position                18944 non-null  int64  
 6   club_joined                  18944 non-null  int64  
 7   nationality_name             18944 non-null  int64  
 8   preferred_foot               18944 non-null  int64  
 9   work_rate                    18944 non-null  int64  
 10  body_type                    18944 non-null  int64  
 11  real_face                    18944 non-null  int64  
 12  ls                           18944 non-null  int64  
 13  st              

# **2. Feature Subsets**

In [13]:
#removing overall from the training data
Y = training_data['overall'] # extract overall
training_data.drop('overall', axis = 1, inplace = True) # drop overall
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 92 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  int64  
 1   long_name                    18944 non-null  int64  
 2   player_positions             18944 non-null  int64  
 3   club_name                    18944 non-null  int64  
 4   league_name                  18944 non-null  int64  
 5   club_position                18944 non-null  int64  
 6   club_joined                  18944 non-null  int64  
 7   nationality_name             18944 non-null  int64  
 8   preferred_foot               18944 non-null  int64  
 9   work_rate                    18944 non-null  int64  
 10  body_type                    18944 non-null  int64  
 11  real_face                    18944 non-null  int64  
 12  ls                           18944 non-null  int64  
 13  st              

In [15]:
#Training using rf for Feature Importance
model=RandomForestRegressor()
model.fit(training_data,Y)

RandomForestRegressor()

In [17]:
#Getting the Important Features from our dataset
name_of_feature=training_data.columns #contains the names of features in our dataset
feature_importance=model.feature_importances_ #contains the scores of each feature

#Sorting the Feature Importance
feature_importance_df=pd.DataFrame({'Feature':name_of_feature,'Importance':feature_importance}) #creates a dataframe with the names of our feature and corresponding scores
feature_importance_df=feature_importance_df.sort_values(by='Importance',ascending=False) #sorts our results from highest to lowest
feature_importance_df



,Feature,Importance
40,value_eur,0.483172
51,release_clause_eur,0.304411
42,age,0.117196
39,potential,0.053997
71,movement_reactions,0.027407
...,...,...
48,weak_foot,0.000019
45,league_level,0.000016
49,skill_moves,0.000010
8,preferred_foot,0.000007


In [18]:
#Using the first 10 features
first_10_features = feature_importance_df['Feature'].values[:11]
first_10_features

array(['value_eur', 'release_clause_eur', 'age', 'potential',
       'movement_reactions', 'wage_eur', 'gk', 'rm', 'lm', 'cb', 'lcb'],
      dtype=object)

In [20]:
#updates to our new training_x value
training_data=training_data[first_10_features]


In [21]:
#scaling the data
sc = StandardScaler()  # scale the data set to be used in the model
training_x = sc.fit_transform(training_data)
scaler2021=sc
training_x = pd.DataFrame(training_x, columns = training_data.columns)
training_x.head(5)

,value_eur,release_clause_eur,age,potential,movement_reactions,wage_eur,gk,rm,lm,cb,lcb
0,13.071234,13.676627,1.655055,3.586563,3.554438,27.845078,0.084727,3.032407,3.032407,-0.395538,-0.395538
1,7.810099,7.310503,2.080838,3.422893,3.664174,10.660644,0.206313,2.974117,2.974117,-0.268013,-0.268013
2,14.045519,13.024735,1.229273,3.259222,3.444701,11.671493,0.084727,2.721528,2.721528,0.278520,0.278520
3,16.773515,16.538836,0.590598,3.259222,3.225227,13.187767,0.206313,3.012977,3.012977,-0.523062,-0.523062
4,16.383801,15.978617,0.803490,3.259222,3.225227,18.242012,0.327900,2.993547,2.993547,1.408023,1.408023


# **3. Machine learning model with cross validation**

In [22]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(training_x,Y,test_size=0.2,random_state=42) #splitting our data

In [23]:
dt = DecisionTreeRegressor(criterion ='squared_error', random_state = 42)
gb = GradientBoostingRegressor(init = dt, n_estimators = 200, learning_rate = 0.01)
XGB_model = xgb.XGBRegressor(objective="reg:squarederror")
rf=RandomForestRegressor(n_estimators=1000, n_jobs = -1)

In [24]:
# Gradient Boosting Regressor
gb.fit(Xtrain,Ytrain)

GradientBoostingRegressor(init=DecisionTreeRegressor(random_state=42),
                          learning_rate=0.01, n_estimators=200)

In [25]:
# Cross validation with the gradient boosting regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth":[5,10, 12],
"n_estimators":[50,70,100],
"max_features":['sqrt', 'log2'],
"criterion":['squared_error', 'friedman_mse'],
"random_state": [20, 22, 24]}
gb_cv = GridSearchCV(estimator = gb,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
gb_cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingRegressor(init=DecisionTreeRegressor(random_state=42),
                                                 learning_rate=0.01,
                                                 n_estimators=200),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [5, 10, 12],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 70, 100],
                         'random_state': [20, 22, 24]},
             scoring='neg_mean_absolute_error')

In [26]:
best_parameters = gb_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the gradient boosting regressor

Best parameters: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 50, 'random_state': 20}


In [27]:
y_pred=gb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.24359989443124835


In [ ]:
# Random Forest Regressor
rf.fit(Xtrain, Ytrain)

RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [ ]:
# Cross validation with the random forest regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth": [10, 12, 15],
"n_estimators": [50, 70, 100],
"criterion": ['squared_error', 'friedman_mse'],
"max_features": ['sqrt', 'log2'],
"random_state": [20, 22, 24]}
rf_cv = GridSearchCV(estimator = rf,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
rf_cv.fit(Xtrain, Ytrain)

KeyboardInterrupt: ignored

In [ ]:
best_parameters = rf_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the random forest regressor

In [ ]:
y_pred=rf_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

In [ ]:
# Cross validation with the XGB regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth": [10, 12, 15],
"random_state": [20, 22, 24]}
xgb_cv = GridSearchCV(estimator = XGB_model,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
xgb_cv.fit(Xtrain, Ytrain)

In [ ]:
best_parameters = xgb_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the XGB regressor

In [ ]:
y_pred=xgb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

# **4. Model Optimization**
Fine tuning , training and re-testing



In [ ]:
training_x.info()

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(training_x,Y,test_size=0.2,random_state=42) #splitting our data

In [ ]:
#cross validation with gb boost which was our best performing model
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth":[7,10,15],
"n_estimators":[50,70,120],
"max_features":['sqrt', 'log2'],
"criterion":['squared_error', 'friedman_mse'],
"random_state": [20, 22, 24]}
gb_cv = GridSearchCV(estimator = gb,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
gb_cv.fit(Xtrain, Ytrain)

In [ ]:
#display the best parameters for the GB regressor
best_parameters = gb_cv.best_params_
print("Best parameters:", best_parameters)

In [ ]:
y_pred=gb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

# **5. Model testing with player 22 data**

In [ ]:
#using specific columns used in our training data in our testing data
new_testing_data = testing_data[training_x.columns]
new_testing_data.info()

In [ ]:
Y = testing_data['overall']
Y

In [ ]:
#grouped columns that are numeric
numeric_testing = new_testing_data.select_dtypes(include=['int', 'float']).columns
numeric_testing = new_testing_data[numeric_testing]
numeric_testing.info()

In [ ]:
#grouped all columns with non-numeric values and object
object_testing = new_testing_data.select_dtypes(exclude=['int', 'float']).columns
object_testing = new_testing_data[object_testing]
object_testing.info()

In [ ]:
#imputed the numeric values using most_frequent ie. to fill in with the mean
imp = SimpleImputer(strategy='mean')
numeric_testing = pd.DataFrame(imp.fit_transform(numeric_testing), columns = numeric_testing.columns)
numeric_testing.info()

In [ ]:
#encode the object data using the label encoder
encoded_object = pd.DataFrame()
for c in object_testing.columns:
  label_encoder = LabelEncoder()
  encoded_object[c] = label_encoder.fit_transform(object_testing[c])
encoded_object.info()

In [ ]:
#rejoined the numeric and non-numeric columns
testing_x = pd.DataFrame()
testing_x = pd.concat([encoded_object, numeric_testing], axis = 1)
testing_x.info()

In [ ]:
sc = StandardScaler()
new_testing_x = sc.fit_transform(testing_x)
new_testing_x = pd.DataFrame(new_testing_x, columns = testing_x.columns)
new_testing_x.head(5)

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(testing_x,Y,test_size=0.2,random_state=42)#split data

In [ ]:
gb = GradientBoostingRegressor(n_estimators = 50, random_state = 20) #using the best perfomind model to train our 2022 data

In [ ]:
gb.fit(Xtrain, Ytrain)

In [ ]:
y_pred = gb.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

# 6. Saving the model using pickle

In [34]:
import pickle

In [ ]:
# deployment_model = '/content/drive/MyDrive/Colab Notebooks/deployment_model.pkl'
# pickle.dump(gb, open(deployment_model, 'wb'))
# loaded_model = pickle.load(open(deployment_model, 'rb'))

In [36]:
with open("deployment_model.pkl", 'wb') as file:
  pickle.dump((gb,scaler2021), file)


In [ ]:
# scaler_model = '/content/drive/MyDrive/Colab Notebooks/scalar_model.pkl'
# pickle.dump(sc,open(scaler_model, 'wb'))
# loaded_model = pickle.load(open(scaler_model, 'rb'))

In [ ]:
with open("scalar_model.pkl", 'wb') as file:
  pickle.dump((sc, file))
